In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

import wrangle

In [38]:
train_scaled, validate_scaled, test_scaled = wrangle.wrangle_split_scale()

In [39]:
train_scaled.head(5)

,Unnamed: 0,bedroomcnt,bathroomcnt,square_feet,taxvaluedollarcnt,taxamount,yearbuilt,zipcode,fips_name,fips_name_Orange,fips_name_Ventura
35112,35112,0.4,0.181818,0.087946,32461,929.72,1926,95982.0,Los Angeles,0,0
12968,12968,0.0,0.000000,0.022846,16714,422.74,1924,95984.0,Los Angeles,0,0
5870,5870,0.4,0.181818,0.096053,233647,2877.34,1953,97040.0,Orange,1,0
37565,37565,0.4,0.272727,0.153374,450261,4748.24,1995,96989.0,Orange,1,0
47416,47416,0.4,0.000000,0.055028,165984,2333.17,1951,96213.0,Los Angeles,0,0


In [40]:
train_scaled.dtypes

Unnamed: 0             int64
bedroomcnt           float64
bathroomcnt          float64
square_feet          float64
taxvaluedollarcnt      int64
taxamount            float64
yearbuilt              int64
zipcode              float64
fips_name             object
fips_name_Orange       uint8
fips_name_Ventura      uint8
dtype: object

### turn it into our train,validate,test sets

In [41]:
# Create the object
# Since taxamount and taxvalue amount are so similar I don't want to move forward with 
# it in my model.
x_train = train_scaled.drop(columns = ['taxvaluedollarcnt', 'taxamount'])
y_train = pd.DataFrame(train_scaled[['taxvaluedollarcnt']])

x_validate = validate_scaled.drop(columns = ['taxvaluedollarcnt', 'taxamount'])
y_validate = pd.DataFrame(validate_scaled[['taxvaluedollarcnt']])

x_test = train_scaled.drop(columns = ['taxvaluedollarcnt', 'taxamount'])
y_train = pd.DataFrame(train_scaled[['taxvaluedollarcnt']])

In [42]:
x_train

,Unnamed: 0,bedroomcnt,bathroomcnt,square_feet,yearbuilt,zipcode,fips_name,fips_name_Orange,fips_name_Ventura
35112,35112,0.4,0.181818,0.087946,1926,95982.0,Los Angeles,0,0
12968,12968,0.0,0.000000,0.022846,1924,95984.0,Los Angeles,0,0
5870,5870,0.4,0.181818,0.096053,1953,97040.0,Orange,1,0
37565,37565,0.4,0.272727,0.153374,1995,96989.0,Orange,1,0
47416,47416,0.4,0.000000,0.055028,1951,96213.0,Los Angeles,0,0
...,...,...,...,...,...,...,...,...,...
10209,10209,0.4,0.181818,0.092941,1977,96941.0,Orange,1,0
24383,24383,0.4,0.181818,0.058631,1991,97330.0,Los Angeles,0,0
24870,24870,0.4,0.181818,0.072470,1957,97318.0,Los Angeles,0,0
2677,2677,0.2,0.000000,0.020144,1927,96282.0,Los Angeles,0,0


In [43]:
y_train

,taxvaluedollarcnt
35112,32461
12968,16714
5870,233647
37565,450261
47416,165984
...,...
10209,519967
24383,106101
24870,63451
2677,343000


In [44]:
y_train['taxvaluedollarcnt'].mean()

512484.22575762786

## Setting up the baseline:
For Regression, we want to decide if we need to use mean or median baseline, so lets look at both

In [45]:
#Create mean baseline model
y_train['baseline_mean_pred'] = y_train['taxvaluedollarcnt'].mean()
y_validate['baseline_mean_pred'] = y_validate['taxvaluedollarcnt'].mean()

#Create median baseline model
y_train['baseline_median_pred'] = y_train['taxvaluedollarcnt'].median()
y_validate['baseline_median_pred'] = y_validate['taxvaluedollarcnt'].median()

In [46]:
#determine RSMSE
results = []

baseline_mean = {
    'model':'baseline_mean',
    'RMSE_train': mean_squared_error(y_train['taxvaluedollarcnt'], y_train['baseline_mean_pred'])**(0.5),
    'RMSE_validate': mean_squared_error(y_validate['taxvaluedollarcnt'], y_validate['baseline_mean_pred'])**(0.5)

}
baseline_median = {
    'model':'baseline_median',
    'RMSE_train': mean_squared_error(y_train['taxvaluedollarcnt'], y_train['baseline_median_pred'])**(0.5),
    'RMSE_validate': mean_squared_error(y_validate['taxvaluedollarcnt'], y_validate['baseline_median_pred'])**(0.5)

}
results.append(baseline_mean)
results.append(baseline_median)

In [47]:
pd.DataFrame(results)

,model,RMSE_train,RMSE_validate
0,baseline_mean,612417.949481,607410.094119
1,baseline_median,628005.785271,623748.462807


# Linear Regression

In [48]:
lr = LinearRegression(normalize = True)

lr.fit(x_train, y_train['taxvaluedollarcnt'])

y_train['lr_train_pred'] = lr.predict(x_train)

y_validate['lr_validate_pred'] = lr.predict(x_validate)

y_train

ValueError: could not convert string to float: 'Los Angeles'

In [ ]:
lr_rmse = {
    'model':'linear regression',
    'RMSE_train': mean_squared_error(y_train['taxvaluedollarcnt'], y_train['lr_train_pred'])**(0.5),
    'RMSE_validate': mean_squared_error(y_validate['taxvaluedollarcnt'], y_validate['lr_validate_pred'])**(0.5)

}
results.append(lr_rmse)
pd.DataFrame(results)

# LassoLars

In [ ]:
lars = LassoLars(alpha=1.0)

lars.fit(x_train, y_train['taxvaluedollarcnt'])

y_train['lars_train_pred'] = lars.predict(x_train)

y_validate['lars_validate_pred'] = lars.predict(x_validate)

y_train

In [ ]:
lars_rmse = {
    'model':'LassoLars regression',
    'RMSE_train': mean_squared_error(y_train['taxvaluedollarcnt'], y_train['lars_train_pred'])**(0.5),
    'RMSE_validate': mean_squared_error(y_validate['taxvaluedollarcnt'], y_validate['lars_validate_pred'])**(0.5)

}
results.append(lars_rmse)
pd.DataFrame(results)

# Polynomial

In [ ]:
# make the polynomial features to get a new set of features
pr = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
x_train_degree2 = pr.fit_transform(x_train)

# transform X_validate_scaled & X_test_scaled
x_validate_degree2 = pr.transform(x_validate)

In [ ]:
# create the model object
lr2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lr2.fit(x_train_degree2, y_train['taxvaluedollarcnt'])

# predict train
y_train['poly_train_pred'] = lr2.predict(x_train_degree2)

# predict validate
y_validate['poly_validate_pred'] = lr2.predict(x_validate_degree2)

In [ ]:
y_train

In [ ]:
poly_rmse = {
    'model':'Polynomial regression',
    'RMSE_train': mean_squared_error(y_train['taxvaluedollarcnt'], y_train['poly_train_pred'])**(0.5),
    'RMSE_validate': mean_squared_error(y_validate['taxvaluedollarcnt'], y_validate['poly_validate_pred'])**(0.5)

}
results.append(poly_rmse)
pd.DataFrame(results)

# TweedieRegressor

In [ ]:
twr = TweedieRegressor()

twr.fit(x_train, y_train['taxvaluedollarcnt'])

y_train['twr_train_pred'] = twr.predict(x_train)

y_validate['twr_validate_pred'] = twr.predict(x_validate)

y_train

In [ ]:
twr_rmse = {
    'model':'TweedieRegressor regression',
    'RMSE_train': mean_squared_error(y_train['taxvaluedollarcnt'], y_train['twr_train_pred'])**(0.5),
    'RMSE_validate': mean_squared_error(y_validate['taxvaluedollarcnt'], y_validate['twr_validate_pred'])**(0.5)

}
results.append(twr_rmse)
pd.DataFrame(results)

In [ ]:
results_df = pd.DataFrame(results)
results_df.sort_values(['RMSE_train', 'RMSE_validate'])